In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [5]:
from pyspark.sql.functions import countDistinct
vine_df.select(countDistinct('review_id')).show()

+-------------------------+
|count(DISTINCT review_id)|
+-------------------------+
|                  2643619|
+-------------------------+



In [6]:
from pyspark.sql.functions import col
filtered_vine_df = vine_df.filter(col('total_votes')>20)
filtered_vine_df.show()
filtered_vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|                Y|
|R34X0ZGVB3YGYO|          5|           47|         47|   N|     

36999

In [7]:
ratio_vine_df = filtered_vine_df.withColumn('ratio',filtered_vine_df['helpful_votes']/filtered_vine_df['total_votes']).filter(col('ratio')>0.5)
ratio_vine_df.show()
ratio_vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|  R39GSNW76GYF|          1|  

35730

In [8]:
paid_vine_df = ratio_vine_df.filter(col('vine') =='Y')
paid_vine_df.show(truncate=False)
paid_vine_df.cache()
paid_vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|ratio             |
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3A71VR1JZD8WF|2          |27           |30         |Y   |N                |0.9               |
|R16OMUJIGI18JZ|5          |72           |72         |Y   |N                |1.0               |
|R3TS8ZP2FHQ9XR|5          |39           |42         |Y   |N                |0.9285714285714286|
|R2MHP919VZN7DI|5          |29           |30         |Y   |N                |0.9666666666666667|
|R1JUJLXQ2CMWKF|4          |25           |26         |Y   |N                |0.9615384615384616|
|R2T7YE0IFI6N9L|3          |24           |25         |Y   |N                |0.96              |
|R25FI3J7WWOYXM|3          |55           |58         |Y   |N                |0.9482758620689655|
|R3P3U4EOWIUD50|3          |27

162

In [9]:
unpaid_vine_df = ratio_vine_df.filter(col('vine') =='N')
unpaid_vine_df.cache()
unpaid_vine_df.show(truncate=False)
unpaid_vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|ratio             |
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|1          |27           |31         |N   |Y                |0.8709677419354839|
|RX9WC9FTIR1XR |5          |25           |25         |N   |Y                |1.0               |
|RGDCOU1KBHMNG |3          |29           |31         |N   |Y                |0.9354838709677419|
|RVTYWID2TPMMY |2          |35           |42         |N   |Y                |0.8333333333333334|
|R2CMPZ5VESGRLY|4          |27           |28         |N   |Y                |0.9642857142857143|
|R3VQPJZ54B55BA|5          |62           |64         |N   |N                |0.96875           |
|R24QM6D7FEDZ5M|2          |36           |43         |N   |Y                |0.8372093023255814|
|R39GSNW76GYF  |1          |20

35568

In [11]:
from pyspark.sql.functions import countDistinct
total_paid_reviews = paid_vine_df.select(countDistinct('review_id')).withColumnRenamed('count(DISTINCT review_id)','total_paid_reviews').show()
total_unpaid_reviews = unpaid_vine_df.select(countDistinct('review_id')).withColumnRenamed('count(DISTINCT review_id)','total_unpaid_reviews').show()


+------------------+
|total_paid_reviews|
+------------------+
|               162|
+------------------+

+--------------------+
|total_unpaid_reviews|
+--------------------+
|               35568|
+--------------------+



In [12]:
total_five_star_paid = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
total_five_star_paid

63

In [13]:
total_five_star_unpaid = unpaid_vine_df.filter(paid_vine_df.star_rating == 5).count()
total_five_star_unpaid

19437

In [14]:
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

162

In [15]:
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

35568

In [16]:
percentage_five_star_paid = total_five_star_paid/total_paid_reviews * 100
percentage_five_star_paid

38.88888888888889

In [17]:
percentage_five_star_unpaid = total_five_star_unpaid/total_unpaid_reviews * 100
percentage_five_star_unpaid

54.64743589743589

In [21]:
from pyspark.sql.functions import avg,col
avg_star_rating_vine = paid_vine_df.agg(avg(col('star_rating'))).show() 
avg_star_rating_vine

+------------------+
|  avg(star_rating)|
+------------------+
|3.9814814814814814|
+------------------+



In [22]:
from pyspark.sql.functions import avg,col
avg_star_rating_non_vine = unpaid_vine_df.agg(avg(col('star_rating'))).show() 
avg_star_rating_non_vine

+------------------+
|  avg(star_rating)|
+------------------+
|3.7702991452991452|
+------------------+

